In [ ]:
# @title Import Dependencies

import time
import hashlib
import os
from datetime import datetime
from io import BytesIO
import random
import pyarrow
import requests
from PIL import Image
import os
import datetime as dt
import os
from datetime import datetime
import pandas
import pandas as pd
import praw
from praw.models import ListingGenerator
from tqdm import tqdm
from adlfs import AzureBlobFileSystem
import os
import hashlib
import os
from datetime import datetime
from io import BytesIO
import random

import requests
from PIL import Image


In [ ]:
# @title Initialize Secrets

os.environ["AZURE_STORAGE_ACCOUNT_NAME"] = "ajdevreddit"

os.environ[
    "AZURE_STORAGE_ACCOUNT_KEY"] = "+9066TCgdeVignRdy50G4qjmNoUJuibl9ERiTGzdV4fwkvgdV3aSVqgLwldgZxj/UpKLkkfXg+3k+AStjFI33Q=="

os.environ[
    "AZURE_STORE_ACCOUNT_KEY"] = "+9066TCgdeVignRdy50G4qjmNoUJuibl9ERiTGzdV4fwkvgdV3aSVqgLwldgZxj/UpKLkkfXg+3k+AStjFI33Q=="

In [ ]:
# @title TODO

class AzureFileStorageAdapter(object):
    def __init__(self, container_name: str = "data"):
        self.__account_name: str = os.environ["AZURE_STORAGE_ACCOUNT_NAME"]
        self.__account_key: str = os.environ["AZURE_STORAGE_ACCOUNT_KEY"]
        self.container_name: str = container_name

    def get_file_storage(self) -> AzureBlobFileSystem:
        return AzureBlobFileSystem(
            account_name=self.__account_name,
            account_key=self.__account_key,
            container_name=self.container_name)

    def get_file_storage_root(self) -> AzureBlobFileSystem:
        return AzureBlobFileSystem(account_name=self.__account_name, account_key=self.__account_key,
                                   container_name=self.container_name)

In [ ]:
# @title TODO

tagging_schema = pyarrow.schema(
    [
        pyarrow.field("id", pyarrow.string()),
        pyarrow.field("subreddit", pyarrow.string()),
        pyarrow.field("author", pyarrow.string()),
        pyarrow.field("title", pyarrow.string()),
        pyarrow.field("caption", pyarrow.string()),
        pyarrow.field("hash", pyarrow.string()),
        pyarrow.field("permalink", pyarrow.string()),
        pyarrow.field("original_url", pyarrow.string()),
        pyarrow.field("image_name", pyarrow.string()),
        pyarrow.field("path", pyarrow.string()),
        pyarrow.field("model", pyarrow.string()),
        pyarrow.field("exists", pyarrow.bool_()),
        pyarrow.field("curated", pyarrow.bool_()),
        pyarrow.field("accept", pyarrow.bool_()),
        pyarrow.field("tags", pyarrow.list_(pyarrow.string())),
        pyarrow.field("azure_caption", pyarrow.string()),
        pyarrow.field("thumbnail_path", pyarrow.string()),
        pyarrow.field("thumbnail_exists", pyarrow.bool_()),
        pyarrow.field("thumbnail_curated", pyarrow.bool_()),
        pyarrow.field("thumbnail_accept", pyarrow.bool_()),
        pyarrow.field("additional_captions", pyarrow.list_(pyarrow.string())),
    ]
)

In [ ]:
# @title TODO

tqdm.pandas(desc="Progress")

file_system: AzureBlobFileSystem = AzureFileStorageAdapter('data').get_file_storage()

In [ ]:
# @title TODO

thumbnail_curation = pandas.read_parquet("data/parquet/thumbnail_curation.parquet", engine='pyarrow', filesystem=file_system, schema=tagging_schema)

display(thumbnail_curation)

In [ ]:
# @title TODO

all_data_from_captions = pandas.read_parquet("data/parquet/all_data_from_captions.parquet", engine='pyarrow', filesystem=file_system)
all_data_from_captions.set_index('id', inplace=True, drop=False)

filtered_data_from_captions = pandas.read_parquet("data/parquet/filtered_data_from_captions.parquet", engine='pyarrow', filesystem=file_system)
filtered_data_from_captions.set_index('id', inplace=True, drop=False)

display(all_data_from_captions)
display(filtered_data_from_captions)

In [ ]:
# @title TODO

import json

display("=== Obtaining Caption Files List ===")
current_captions = file_system.ls("data/caption")
display(f"current caption files: {len(current_captions)}")

all_data = []
filtered_data = []

for caption_file in tqdm(current_captions, total=len(current_captions), desc='Reading caption files'):
    image_id = caption_file.split('/')[-1].split('.')[0]
    if image_id in all_data_from_captions['id'].values or image_id in filtered_data_from_captions['id'].values:
        continue
    try:
        file_size = file_system.size(caption_file)
        if file_size == 0:
            display(f'Empty file -- removing {image_id}', clear=True)
            file_system.rm(caption_file)
            continue
        caption_data = json.loads(file_system.read_text(caption_file, encoding='utf-8'))
        caption_data["id"] = image_id
        dense_caption_result = caption_data.get('denseCaptionsResult')
        metadata = caption_data.get('metadata')
        tags_result = caption_data.get('tagsResult')
        smart_crop_result = caption_data.get('smartCropsResult')
        basic_caption = caption_data.get('captionResult')
        _filtered_data = {
            "id": image_id,
            "captions": [basic_caption],
            "dense_captions": dense_caption_result['values'],
            "meta": [metadata],
            "tags": tags_result['values'],
            "smart_crop": smart_crop_result['values']
        }
        all_data.append(caption_data)
        filtered_data.append(_filtered_data)
    except Exception as e:
        display(f" Exception {e} for {image_id}", clear=True)
        continue

In [ ]:
# @title TODO

new_all_data_from_captions = pandas.DataFrame(data=all_data)

new_filtered_data_from_captions = pandas.DataFrame(data=filtered_data)

display(new_all_data_from_captions)

display(new_filtered_data_from_captions)

combined_new_all_data_from_captions = pandas.concat([all_data_from_captions, new_all_data_from_captions])

combined_new_filtered_data_from_captions = pandas.concat([filtered_data_from_captions, new_filtered_data_from_captions])

display(combined_new_all_data_from_captions)

display(combined_new_filtered_data_from_captions)

combined_new_all_data_from_captions.to_parquet("data/parquet/all_data_from_captions.parquet", engine='pyarrow', filesystem=file_system)

combined_new_filtered_data_from_captions.to_parquet("data/parquet/filtered_data_from_captions.parquet", engine='pyarrow', filesystem=file_system)

In [ ]:
# @title TODO

all_data_from_captions = pandas.read_parquet("data/parquet/all_data_from_captions.parquet", engine='pyarrow', filesystem=file_system)
display(all_data_from_captions)

filtered_data_from_captions = pandas.read_parquet("data/parquet/filtered_data_from_captions.parquet", engine='pyarrow', filesystem=file_system)

display(filtered_data_from_captions)

In [ ]:
# @title TODO


tags = pandas.DataFrame({'id': filtered_data_from_captions.id, 'tags': filtered_data_from_captions.tags})
out = []
for i, r in tqdm(tags.iterrows(), total=len(tags)):
    if r['tags'] is None:
        continue
    tag = pandas.json_normalize(r['tags'])
    tag['id'] = r['id']
    d = tag.to_dict(orient='records')
    out.extend(d)
converted_tags = pandas.DataFrame(data=out)
display(converted_tags)
converted_tags.to_parquet("data/parquet/tags.parquet", engine='pyarrow', filesystem=file_system)

In [ ]:
# @title TODO

dense = pandas.DataFrame({'id': filtered_data_from_captions.id, 'dense_captions': filtered_data_from_captions.dense_captions})

out = []
for i, r in tqdm(dense.iterrows(), total=len(dense)):
    if r['dense_captions'] is None:
        continue
    dense_caption = pandas.json_normalize(r['dense_captions'])
    dense_caption['id'] = r['id']
    d = dense_caption.to_dict(orient='records')
    out.extend(d)

converted = pandas.DataFrame(data=out)
display(converted)
converted.to_parquet("data/parquet/dense_captions.parquet", engine='pyarrow', filesystem=file_system)

In [ ]:
# @title TODO

single_caption_data = pandas.DataFrame(
    {
        'id': all_data_from_captions['id'],
        'azure_caption': [item['text'] for item in all_data_from_captions['captionResult']],
        'tags': [[foo['name'] for foo in item['values']] for item in all_data_from_captions['tagsResult']]
    })

single_caption_data_indexed = single_caption_data.set_index("id")

for index, row in tqdm(single_caption_data_indexed.iterrows(), total=len(single_caption_data_indexed)):
    print(index)

display("== Updated With Basic Captions ==")
display(thumbnail_curation)

In [ ]:
# @title TODO

%%time



accepted_final.to_parquet("data/parquet/curation_2.parquet", engine='pyarrow', filesystem=file_system, schema=tagging_schema)
del accepted_final

In [ ]:
# @title TODO

%%time



accepted_final = pandas.read_parquet("data/parquet/curation_2.parquet", engine='pyarrow', filesystem=file_system, schema=tagging_schema)
display(accepted_final)

In [ ]:
# @title TODO

%%time



def get_aspect_ratio(x: object):
    return x['crops'][0]['aspectRatio']


def get_bounding_box(x: object):
    return x['crops'][0]['boundingBox']

In [ ]:
# @title TODO

%%time

cropping = pandas.DataFrame(
    {'id': filtered_data_from_captions['id'], 'crops': filtered_data_from_captions['smart_crop']}).set_index('id',
                                                                                                             drop=False)

cropping['aspectRatio'] = cropping.progress_apply(lambda x: get_aspect_ratio(x), axis=1)
cropping['bounding_box'] = cropping.progress_apply(lambda x: get_bounding_box(x), axis=1)
cropping['x'] = cropping.progress_apply(lambda x: x['bounding_box']['x'], axis=1)
cropping['y'] = cropping.progress_apply(lambda x: x['bounding_box']['y'], axis=1)
cropping['w'] = cropping.progress_apply(lambda x: x['bounding_box']['w'], axis=1)
cropping['h'] = cropping.progress_apply(lambda x: x['bounding_box']['h'], axis=1)

display(cropping)

In [ ]:
# @title TODO

%%time


def create_thumbnail(target_image_id: str, file_names: list, crops: pandas.DataFrame, curated_data: pandas.DataFrame):
    _file_system: AzureBlobFileSystem = AzureFileStorageAdapter('data').get_file_storage()

    out_path = f"data/image/thumbnail/{target_image_id}.jpg"
    try:
        if target_image_id is None or out_path in file_names:
            # print(f'Image {target_image_id} already exists, skipping')
            return out_path

        cropping_information = cropping.loc[crops['id'] == target_image_id]
        if cropping_information is None or len(cropping_information) == 0:
            # print(f'No cropping information for {target_image_id}, skipping')
            return "/data/nope"

        record = curated_data.loc[curated_data['id'] == target_image_id]
        record_path = record.to_dict(orient='records')[0]['path']
        image_url = file_system.url(record_path)
        original_image = Image.open(requests.get(image_url, stream=True).raw)
        copied_image = original_image.copy()
        original_image.close()

        cropped = copied_image.crop((cropping_information['x'].values[0],
                                     cropping_information['y'].values[0],
                                     cropping_information['x'].values[0] +
                                     cropping_information['w'].values[0],
                                     cropping_information['y'].values[0] +
                                     cropping_information['h'].values[0]))
        copied_image.close()

        resized = cropped.resize((512, 512), 1)
        resized.save('temp.jpg')
        resized.close()
        file_system.upload('temp.jpg', out_path, overwrite=True)
        print(f'Thumbnail created for {target_image_id}')
        return out_path

    except Exception as ex:
        print(f'Error creating thumbnail for {target_image_id}: {ex}')
        return "/data/nope"

In [ ]:
# @title TODO

%%time


file_names = file_system.ls('data/image/thumbnail')

accepted_final['thumbnail_path'] = accepted_final.progress_apply(
    lambda x: create_thumbnail(x['id'], file_names, cropping, accepted_final), axis=1)

display(accepted_final)

In [ ]:
# @title TODO

%%time



accepted_final['thumbnail_exists'] = accepted_final.progress_apply(lambda x: file_system.exists(x['thumbnail_path']),
                                                                   axis=1)

display(accepted_final)

In [ ]:
# @title TODO

%%time



accepted_copy = accepted_final.copy()

accepted_copy['additional_captions'] = accepted_copy.progress_apply(lambda x: [], axis=1)

accepted_slice = accepted_copy.loc[accepted_copy['thumbnail_exists'] == True, tagging_schema.names]

accepted_slice.dropna(inplace=True)

accepted_slice.reset_index(inplace=True, drop=True)

display(accepted_slice)

In [ ]:
# @title TODO

%%time


current_data.set_index('id', inplace=True, drop=False)
accepted_slice.set_index('id', inplace=True, drop=False)

filtered = accepted_slice.loc(accepted_slice['id'].isnotin(current_data['id']))
filtered.dropna(inplace=True)
filtered.reset_index(inplace=True, drop=True)
display(filtered)